# Optimization (tutorial)

In this tutorial you will learn to code and use common optimization algorithms for static models.

In [ ]:
]add NLsolve

   Updating registry at `C:\Users\phoeb\.juliapro\JuliaPro_v1.4.0-1\registries\JuliaPro`


In [ ]:
]add Optim

In [ ]:
]add Plots


---

## Profit optimization by a monopolist

A monopolist produces quantity $q$ of goods X at price $p$. Its cost function is $c(q) = 0.5 + q (1-qe^{-q})$

The consumer's demand for price $p$ is $x(p)=2 e^{-0.5 p}$ (constant elasticity of demand to price).

__Write down the profit function of the monopolist and find the optimal production (if any). Don't use any library except for plotting.__



We write the profit function:
$\pi(p) = px(p) - c(x(p)) = 2pe^{-0.5p} - 0.5 -  2e^{-0.5p}(1- 2e^{-0.5p}e^{- 2e^{-0.5p}}) = 2(p-1)e^{-0.5p} + 4e^{-p + 2e^{0.5p}} - 0.5$

In [5]:
using Plots
x = 1:10; y = 2*(x-1)*e**(-0.5*x) + 4*e**(-x+2*e**(0.5*x)) - 0.5; 
plot(x,y)

ArgumentError: ArgumentError: Package Plots not found in current path:
- Run `import Pkg; Pkg.add("Plots")` to install the Plots package.


The optimal choice for the monopolist is price $\rightarrow \infty$, which means production $\rightarrow 0$.<br>
Indeed, we restrict ourselves to $p>0$, and rewrite the profit function:<br>
$$\pi(p) = 2(p-1)e^{-0.5p} + 4e^{-p + 2e^{0.5p}} - 0.5 = e^{-0.5p} (2(p-1) + 4e^{-0.5p + 2e^{0.5p}})$$
First, note that profit is always positive for $p>1$. Also, see that $2p$ grows much faster than $e^{-0.5p}$ decreases, so $(2(p-1) + 4e^{-0.5p + 2e^{0.5p}})$ grows much faster too and on the whole, profit is increasing. Then, it is impossible to have FOC$ = 0$, and there is no maximum besides $p\rightarrow \infty$.

---

## (modified) Solow model

This model tries to explain long-term productivity, its relation to productivity and the speed of convergence.

The setup is the following:

- a country accumulates capital $k_t$
- population $n_t$ grows at rate $g$
- capital and labour are combined to produce $y_t = A F(k_t, n_t)$ where $A$ is total factor productivity.
- capital depreciates at rate $\delta$. Its law of motion is $k_{t+1}=k_{t}+i_{t}$ where $i_t$ is the amount invested at time $t$
- production is either consumed or invested: $y_t = c_t + i_t$

We choose a Cobb-Douglas specification with constant returns to scale $F(k_t, n_t) = k_t^{\alpha} n_t^{1-\alpha}$.

In the Ramsey–Cass–Koopmans  model (also called neoclassical growth model), a representative agent would choose $c_t$ in every period so as to maximize an intertemporal utility like $\sum_{t \geq 0} \beta^t U(c_t)$ where $\beta \in [0,1[$ is a time discount and $U(x)=\frac{x^{1-\gamma}}{1-\gamma}$ is the instantaneous felicity.

Here we follow the Solow-Swan specification and assume instead there is a fixed fraction $s\in[0,1[$ of income in every period, which is saved and invested in every period. In other words:

$$i_t = s y_t$$

As a result, the dynamic of capital and all other variables will be backward looking and can be simulated easily.

__Calibrate parameters $\beta$, $\delta$ and $g$, that is, propose plausible values for them, from the litterature, or by matching some observable fact.__

https://www150.statcan.gc.ca/n1/pub/15-206-x/2015039/t/tbl06-eng.htm, https://www150.statcan.gc.ca/n1/pub/15-206-x/2015039/t/tbl07-eng.htm.

Take $\delta = 0.05$, a "generous" estimate

https://mpra.ub.uni-muenchen.de/39736/1/MPRA_paper_39736.pdf

Take $\beta = 0.96$, the USA estimate.

https://www.google.com/search?q=population+growth+usa&rlz=1C1JZAP_enFR898FR898&oq=population+growth+usa&aqs=chrome..69i57j0l7.4688j0j7&sourceid=chrome&ie=UTF-8

Take $g = 0.011$ the world average population growth rate

In [6]:
δ = 0.05
β = 0.96
g = 0.011

0.011

__Detrend the equations of the model w.r.t. population growth. Denote the detrended variables with a hat (in the code we assume all variables are detrended and ignore the hat).__

$$\hat{k}_t = \frac{k_t}{n_t}$$


$$\hat{y}_t = \frac{y_t}{n_t} = A\frac{k_t^{\alpha}n_t^{1-\alpha}}{n_t} = A \frac{k_t^{\alpha}n_t^{1-\alpha}}{n_t^{\alpha}n_t^{1-\alpha}} = A({\frac{k_t}{n_t}})^{\alpha} = \hat{k}_t^{\alpha}$$

$$\hat{k}_{t+1} = \frac{k_{t+1}}{n_{t+1}} = \frac{k_t(1-\delta) + sy_t}{n_{t+1}} = (1-\delta) \frac{k_t}{n_t} \frac{n_t}{n_{t+1}} + s \frac{y}{n_t}\frac{n_t}{n_{t+1}} = \frac{1}{1+g} ( (1-\delta) \frac{k_t}{n_t} + s \frac{y}{n_t}) = \frac{(1-\delta) \hat{k}_t + s \hat{y}_t}{1+g}  = \frac{(1-\delta) \hat{k}_t + s {\hat{k}_t}^{\alpha}}{1+g} $$


Our goal is to compute the steady-state and assess its stability.

__Compute a function `f` which returns the capital $\hat{k}_{t+1}$ as a function of $\hat{k_t}$__

In [7]:
function f(k_t,s,α)
    return ((1-δ)*k_t + s*(k_t)^(α))/(1+g)
end

f(5,0.1,0.5)

4.919492381552898

__Starting from an initial level $\hat{k}_0$ compute successive iterates of $f$ to find the long-run level $\overline{k}$ of capital per capita. (Bonus: produce a nice plot of the convergence)__

In [41]:
function lr(k_start,s,α)
    k_new = f(k_start,s,α)
    k=[k_start, k_new]
    it = [0,1]
    newit = 1
    while abs( k_new - k_start) > 0.0001
        newit = newit + 1
        #println(k_new)
        k_start = k_new
        k_new = f(k_start,s,α)
        #print(k)
        k = [k;[k_new]]
        it = [it;[newit]]
    end
    return [k_new, k,it, newit]
end

result = lr(5,0.1,0.5)

print(result[1], ", convergence: ", result[4])

plot(result[2], result[3])

0.8133261413782925, convergence: 128

UndefVarError: UndefVarError: plot not defined

__What factors affect the steady-state level of capital?__

Most saliently, the saving rate. Although every parameter is significant:

In [42]:
δ = 0.05
g = 0.011

basic = lr(5,0.1,0.5)[1]

print("s = 0.1: ", basic, " ---> s = 0.3: ",lr(5,0.3,0.5)[1], "\n")

print("α = 0.5: ", basic, " ---> α = 0.33: ",lr(5,0.3,0.33)[1], "\n")

g = 0.02
print("g = 0.011: ", basic, " ---> g = 0.2: ",lr(5,0.1,0.5)[1], "\n")

g = 0.011
δ = 0.1
print("δ = 0.05: ", basic, " ---> δ = 0.1: ",lr(5,0.1,0.5)[1], "\n")

s = 0.1: 2.690581164129932 ---> s = 0.3: 24.183869373929877
α = 0.5: 2.690581164129932 ---> α = 0.33: 10.77530360036766
g = 0.011: 2.690581164129932 ---> g = 0.2: 2.043623655040326
δ = 0.05: 2.690581164129932 ---> δ = 0.1: 0.8133261413782925


__Study the stability of $f$ around $\overline{k}$. Which factors affect the speed of convergence towards the steady-state?__

In [43]:
δ = 0.05
g = 0.011

basic = lr(5,0.1,0.5)[4]

print("s = 0.1: ", basic, " ---> s = 0.3: ",lr(5,0.3,0.5)[4], "\n")

print("α = 0.5: ", basic, " ---> α = 0.33: ",lr(5,0.3,0.33)[4], "\n")

g = 0.02
print("g = 0.011: ", basic, " ---> g = 0.2: ",lr(5,0.1,0.5)[4], "\n")

g = 0.011
δ = 0.1
print("δ = 0.05: ", basic, " ---> δ = 0.1: ",lr(5,0.1,0.5)[4], "\n")

s = 0.1: 210 ---> s = 0.3: 295
α = 0.5: 210 ---> α = 0.33: 192
g = 0.011: 210 ---> g = 0.2: 192
δ = 0.05: 210 ---> δ = 0.1: 128


__Compute the steady-state directly using a Newton method and compare convergence speed.__

In [44]:
function fun(x,s,a)
    return f(x,s,a) - x
end

fun (generic function with 1 method)

In [45]:

function Newton(k_start,s,α)
    k_new = fun(k_start,s,α)
    k=[k_start, k_new]
    it = [0,1]
    newit = 1
    while abs(k_new - k_start) > 0.0001 && abs(fun(k_start,s,α)) > 0.0001
        newit = newit + 1
        #println(k_new)
        k_start = k_new
        k_new = fun(k_start,s,α)
        #print(k)
        k = [k;[k_new]]
        it = [it;[newit]]
    end
    return [k_new, k,it, newit]
end

result = lr(5,0.1,0.5)

print(result[1], ", convergence: ", result[4])

plot(result[2], result[3])


0.8133261413782925, convergence: 128

UndefVarError: UndefVarError: plot not defined

Apparently, the convergence is as fast.

__Suppose one tries to maximize steady-state consumption by choosing saving rate $s$. Which value would one choose?__

In [46]:
s = 0: 0.99 ; res = Newton(5,s,0.5)
plot(s,res)

MethodError: MethodError: no method matching +(::Float64, ::StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}})
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:529
  +(::Float64, !Matched::Float64) at float.jl:401
  +(::AbstractFloat, !Matched::Bool) at bool.jl:106
  ...

__(Bonus) Suppose an agent is given the intertemporal utility from the Ramsey–Cass–Koopmans model but chooses saving rate once for all. Which saving rate would one choose? Is it the same for all initial levels of capital?__



---

## Exercise: constrained optimization

Consider the function $f(x,y) = 1-(x-0.5)^2 -(y-0.3)^2$.

__Use Optim.jl to minimize $f$ without constraint. Check you understand diagnositc information.__

In [1]:
#using Pkg; Pkg.add("Optim")



f(x)= 1 -(x[1] - 0.5)^2 - (x[2] - 0.3)^2

x0 = [0.0,0.0]
results = optimize(f,x0)

print(results)

LoadError: syntax: unexpected "]"

__Now, consider the constraint $x<0.3$ and maximize $f$ under this new constraint.__

In [ ]:
using Optim

function g!(G, x)
G[1] = -2.0 * x[1] + 1
G[2] = -2.0 * y + 0.6
end

lower = [-Inf, -Inf]
upper = [0.3, Inf]
initial_x = [2.0, 2.0]
inner_optimizer = GradientDescent()
results = optimize(f, g!, lower, upper, initial_x, Fminbox(inner_optimizer))

print(results)

__Reformulate the problem as a root finding with lagrangians. Write the complementarity conditions.__

The Lagrangian is:

$L(x,y,\lambda) = 1 - (x-0.5)^2 - (y-0.3)^2 - \lambda x$

And the conditions are:

$\frac{d L}{dx} = 0 \text{ i.e. } -2x + 1 = 0$

$\frac{d L}{dy} = 0 \text{ i.e. } -2x + 0.6 = 0$

$\lambda(x-0.3) = 0$


__Solve using NLSolve.jl__

In [ ]:
using NLsolve

function f!(F, x)
    F[1] = -2*x[1] + 1
    F[2] = -2*y + 0.6
    F[3] = x[3] *(x[1] - 0.3)
end

function j!(J, x)
    J[1, 1] = -2
    J[1, 2] = 0
    J[1, 3] = 0
    J[2, 1] = 0
    J[2, 2] = -2
    J[2, 3] = 0
    J[3, 1] = x[3]
    J[3, 2] = 0
    J[3, 3] = x[1] - 0.3
end


results = nlsolve(f!, j!, [ 0.0; 1.0])

print(results)

---

## Consumption optimization

A consumer has preferences $U(c_1, c_2)$ over two consumption goods $c_1$ and $c_2$.

Given a budget $I$, consumer wants to maximize utility subject to the budget constraint $p_1 c_1 + p_2 c_2 \leq I$.

We choose a Stone-Geary specification where

$U(c_1, c_2)=\beta_1 \log(c_1-\gamma_1) + \beta_2 \log(c_2-\gamma_2)$

__Write the Karush-Kuhn-Tucker necessary conditions for the problem.__

Complementary slackness:<br>
$$\lambda \geq 0$$
$$\lambda(p_1 c_1 + p_2 c_2 - I) = 0$$
First-order conditions:
$$\beta_1 \frac{1}{c_1 - \gamma_1} - \lambda p_1 =0$$
$$\beta_2 \frac{1}{c_2 - \gamma_2} - \lambda p_2 =0$$

__Verify the KKT conditions are sufficient for optimality.__

Yes, as $U$ is concave, and $p_1 c_1 + p_2 c_2 - I$ is continuously differentiable and convex.

__Derive analytically the demand functions, and the shadow price.__

$\beta_1 \frac{1}{c_1 - \gamma_1} - \lambda p_1 =0 \iff \lambda p_1 (c_1 - \gamma_1) = \beta_1 \iff c_1 = \frac{\beta_1}{\lambda p_1} + \gamma_1$. 

Likewise, $\beta_2 \frac{1}{c_2 - \gamma_2} - \lambda p_2 =0  \iff c_2 = \frac{\beta_2}{\lambda p_2} + \gamma_2$.

$\lambda(p_1c_1 + p_2c2) = 0 \iff \lambda(\frac{\beta_1}{\lambda p_1} + \gamma_1 + \frac{\beta_2}{\lambda p_2} + \gamma_2) \iff \lambda = \frac{-\beta_1 - \beta_2}{p_1\gamma_1 + p_2 \gamma_2}$

Which then gives:

$c_1 = \frac{\beta_1 (p_1\gamma_1 + p_2 \gamma_2)}{p_1(-\beta_1 - \beta_2)} + \gamma_1$

$c_2 = \frac{\beta_2(p_1\gamma_1 + p_2 \gamma_2)}{p_2 (-\beta_1 - \beta_2)} + \gamma_2$.

__Interpret this problem as a complementarity problem and solve it using NLSolve__

In [ ]:
using NLsolve

function f!(F, x)
    F[1] = -2*x[1] + 1
    F[2] = -2*y + 0.6
    F[3] = x[3] *(x[1] - 0.3)
end

function j!(J, x)
    J[1, 1] = -2
    J[1, 2] = 0
    J[1, 3] = 0
    J[2, 1] = 0
    J[2, 2] = -2
    J[2, 3] = 0
    J[3, 1] = x[3]
    J[3, 2] = 0
    J[3, 3] = x[1] - 0.3
end


results = nlsolve(f!, j!, [ 0.0; 1.0])

print(results)

__Produce some nice graphs with isoutility curves, the budget constraint and the optimal choice.__

